<a href="https://colab.research.google.com/github/abdou70/NLP_WEEK_1_lab_1_AMMI/blob/main/ABDOU_NIANG_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*The* goal of this lab is to implement a language identifier (LID).

Our first model will be based on Naive Bayes.

In [96]:
import io, sys, math, re
from collections import defaultdict

The next function is used to load the data. Each line of the data consist of a label (corresponding to a language), followed by some text, written in that language. Here is an example of data:

```__label__de Zur Namensdeutung gibt es mehrere Varianten.```


In [97]:
def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

You can now try loading the first dataset `train1.txt` and look what examples look like.

In [98]:
# data = load_data("train1.txt")
# print(data)
# print(data[1])

[('__label__de', ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.']), ('__label__de', ['Tom', 'ist', 'an', 'Kunst', 'völlig', 'uninteressiert.']), ('__label__hu', ['Végeztem', 'Tomival.']), ('__label__de', ['„Wird', 'das', 'in', 'der', 'Werkstatt', 'gemacht?“', '–', '„Nein,', 'das', 'muss', 'an', 'Ort', 'und', 'Stelle', 'erledigt', 'werden.“']), ('__label__ru', ['У', 'меня', 'есть', 'яблоко.']), ('__label__it', ['Non', 'possiamo', 'lasciarle', 'lì.']), ('__label__ru', ['Том', 'считает,', 'что', 'школа', '—', 'это', 'пустая', 'трата', 'времени.']), ('__label__en', ['My', 'fathers', "don't", 'speak', 'Dutch.']), ('__label__es', ['El', 'niño', 'no', 'sabe', 'cómo', 'comportarse.']), ('__label__ru', ['Она', 'думала,', 'что', 'он', 'переночует', 'у', 'неё.']), ('__label__tr', ['Helikopter', 'neden', 'kentin', 'üstünde', 'uçuyor?']), ('__label__it', ['Lo', 'prenda!', 'È', 'un', 'proiettile', 'magico.']), ('__label__eo', ['Mi', 'volas', 'novan', 'poŝtelefonon!']), ('__label_

In [99]:
print(len(data))

10000


Next, we will start implementing the Naive Bayes method. This technique is based on word counts, and we thus need to start by implementing a function to count the words and labels of our training set.

`n_examples` is the total number of examples

`n_words_per_label` is the total number of words for a given label

`label_counts` is the number of times a given label appears in the training data

`word_counts` is the number of times a word appears with a given label

In [89]:
# def count_words(data):
#     n_examples = 0
#     # n_words_per_label = defaultdict(lambda: 0)
#     n_words_per_label ={}
#     label_counts = defaultdict(lambda: 0)
#     #word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
#     tab=[]
#     tab1=[]
#     tab2=[]
#     word_counts={}
#     de=es=en=eo=pt=tr=fr=it=ru=hu=0
#     tab_de=tab_es=tab_en=tab_eo=tab_pt=tab_tr=tab_fr=tab_it=tab_ru=tab_hu=[]
#     for example in data:
#         n_examples+=1
#         label, sentence = example
#         # print('my label and sentence',label,':',sentence)
#         if label=="__label__de":
#           de=de+len(sentence)
#           for j in sentence:
#             tab_de.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_de}
#         elif label=="__label__es":
#           es=es+len(sentence)
#           for j in sentence:
#             tab_es.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_es}
#         elif label=="__label__en":
#           en=en+len(sentence)
#           for j in sentence:
#             tab_en.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_en}
#         elif label=="__label__eo":
#           eo=eo+len(sentence)
#           for j in sentence:
#             tab_eo.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_eo}
#         elif label=="__label__pt":
#           pt=pt+len(sentence)
#           for j in sentence:
#             tab_pt.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_pt}  
#         elif label=="__label__tr":
#           tr=tr+len(sentence)
#           for j in sentence:
#             tab_tr.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_tr}  
#         elif label=="__label__fr":
#           fr=fr+len(sentence)
#           for j in sentence:
#             tab_fr.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_fr}  
#         elif label=="__label__it":
#           it=it+len(sentence)
#           for j in sentence:
#             tab_it.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_it}  
#         elif label=="__label__ru":
#           ru=ru+len(sentence)
#           for j in sentence:
#             tab_ru.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_ru}  
#         elif label=="__label__hu":
#           hu=hu+len(sentence)
#           for j in sentence:
#             tab_hu.append(j)
#           word_counts[label]={j:tab_de.count(j) for j in tab_hu}  
#         tab.append(label)
#         tab1.append(label)
#     # print('000',tab)
#     # print('&&&',tab1)
#     for i in tab:
#       if i not in tab2:
#         tab2.append(i)
#     # print(tab)
#     # print(tab2)
#     xx=[de,hu,ru,it,en,es,tr,eo,pt,fr]
#     # print(type(xx))
#     for i in enumerate(tab2):
#       n_words_per_label[i[1]]=xx[i[0]]
#     label_counts={i:tab.count(i) for i in tab}
#     return {'label_counts': label_counts, 
#             'word_counts': word_counts, 
#             'n_examples': n_examples, 
#             'n_words_per_label': n_words_per_label}

In [100]:
def count_words(data):
    n_examples = 0
    n_words_per_label = defaultdict(lambda: 0)
    # n_words_per_label ={}
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    tab=[]
    tab1=[]
    tab2=[]
    # word_counts={}
    # de=es=en=eo=pt=tr=fr=it=ru=hu=0
    # tab_de=tab_es=tab_en=tab_eo=tab_pt=tab_tr=tab_fr=tab_it=tab_ru=tab_hu=[]
    for example in data:
        n_examples+=1
        label, sentence = example
        for index_word , words_in in enumerate(sentence):
            n_words_per_label[label] += 1
            word_counts[label][words_in] += 1.0  
        tab.append(label)
    label_counts={i:tab.count(i) for i in tab}
    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'n_examples': n_examples, 
            'n_words_per_label': n_words_per_label}

In [101]:
count_words(data)

{'label_counts': {'__label__de': 828,
  '__label__en': 2137,
  '__label__eo': 1020,
  '__label__es': 564,
  '__label__fr': 650,
  '__label__hu': 432,
  '__label__it': 1327,
  '__label__pt': 578,
  '__label__ru': 1271,
  '__label__tr': 1193},
 'n_examples': 10000,
 'n_words_per_label': defaultdict(<function __main__.count_words.<locals>.<lambda>>,
             {'__label__de': 6630,
              '__label__en': 16444,
              '__label__eo': 7647,
              '__label__es': 3927,
              '__label__fr': 4718,
              '__label__hu': 2271,
              '__label__it': 7759,
              '__label__pt': 4044,
              '__label__ru': 7387,
              '__label__tr': 6026}),
 'word_counts': defaultdict(<function __main__.count_words.<locals>.<lambda>>,
             {'__label__de': defaultdict(<function __main__.count_words.<locals>.<lambda>.<locals>.<lambda>>,
                          {'Ich': 140.0,
                           'würde': 9.0,
                           

Next, using the word and label counts from the previous function, we can implement the prediction function.

Here, `mu` is a regularization parameter (Laplace smoothing), and `sentence` is the list of words corresponding to the test example.

In [102]:
def predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label):
    best_label = None
    best_score = float('-inf')

    for label in word_counts.keys():
        score = 0.0
        ## FILE CODE
        voc_size= len(word_counts[label])
        for w in sentence:
          word_count_w= word_counts[label][w] + mu
          denominator= n_words_per_label[label] + voc_size*mu
          score+= math.log(word_count_w/denominator)
          if score > best_score:
            best_score=score
            best_label= label
    return best_label

The next function will be used to evaluate the Naive Bayes model on a validation set. It computes the accuracy for a particular regularization parameter `mu`.

In [103]:
def compute_accuracy(valid_data, mu, counts):
    accuracy = 0.0
    for label, sentence in valid_data:
        ## FILL CODE
        pred= predict(sentence, mu, **counts)  
        if pred == label:
          accuracy+=1.0
    return (accuracy/float(len(valid_data))) * 100.00

In [104]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("train1.txt")
valid_data = load_data("valid1.txt")
counts = count_words(train_data)

print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")


** Naive Bayes **

Validation accuracy: 69.700

